In [1]:
import pandas as pd 
import numpy as np 
import torch 
import matplotlib.pyplot as plt 
import torch 


In [2]:
print(torch.__version__)
data = pd.read_csv("/home/jik19004/FilesToRun/ASOS_10_CT_stations_tmpc_demand_2011_2023.csv").drop(columns=["Unnamed: 0"])
data.head()

1.8.1+cu102


,Datetime,BDL_tmpc,BDR_tmpc,DXR_tmpc,GON_tmpc,HFD_tmpc,HVN_tmpc,IJD_tmpc,MMK_tmpc,OXC_tmpc,SNC_tmpc,Demand
0,1/1/2011 0:00,1.11,1.11,0.00,2.78,1.67,1.11,-2.78,-1.67,3.0,2.0,3053.0
1,1/1/2011 1:00,1.11,2.22,-0.56,3.33,2.22,3.33,-2.78,-1.67,3.0,3.0,2892.0
2,1/1/2011 2:00,-0.56,2.78,-1.67,3.33,2.78,2.78,-2.22,0.56,3.0,3.0,2774.0
3,1/1/2011 3:00,-1.11,2.22,-1.11,3.89,2.22,1.11,-2.78,0.56,2.0,4.0,2710.0
4,1/1/2011 4:00,-1.67,1.67,-1.11,3.33,2.22,-0.56,-2.78,-1.67,4.0,5.0,2698.0


In [3]:
from datetime import datetime 

WeatherData = pd.read_csv('/home/jik19004/FilesToRun/ASOS_10_CT_stations_tmpc_demand_2011_2023.csv').drop(columns = ["Unnamed: 0"])
WeatherData.ffill(inplace = True)
WeatherData.bfill(inplace = True) # fill in missing values with the previous value.
DateTimeCol = WeatherData["Datetime"]
HourCol = []
WeekDayorWeekEndCol = [] 

for date in DateTimeCol:
    date = datetime.strptime(date, "%m/%d/%Y %H:%M")
    HourCol.append(date.hour)
    if date.weekday() < 5:
        WeekDayorWeekEndCol.append(0)
    else:
        WeekDayorWeekEndCol.append(1)

WeatherData.drop(columns = ["Datetime"], inplace = True) # drop the datetime column. 


WeatherData.insert(0, "Hour", HourCol)
WeatherData.insert(1, "Weekday or Weekend", WeekDayorWeekEndCol)
DateTimeCol = [datetime.strptime(date, "%m/%d/%Y %H:%M") for date in DateTimeCol]


In [ ]:
train_start = "1/1/2017 0:00"
train_end = "12/31/2018 23:00"

val_start = "1/1/2019 0:00"
val_end = "12/31/2019 23:00"

test_start = "1/01/2020 0:00"
test_end = "12/31/2021 0:00"


date_train_start = datetime.strptime(train_start, "%m/%d/%Y %H:%M")
date_train_end = datetime.strptime(train_end, "%m/%d/%Y %H:%M")

date_val_start = datetime.strptime(val_start, "%m/%d/%Y %H:%M")
date_val_end = datetime.strptime(val_end, "%m/%d/%Y %H:%M")

date_test_start = datetime.strptime(test_start, "%m/%d/%Y %H:%M")
date_test_end = datetime.strptime(test_end, "%m/%d/%Y %H:%M")

DateTimeCol_num = np.array(DateTimeCol)

print(np.where(DateTimeCol_num == date_train_start)[0][0])
print(np.where(DateTimeCol_num == date_train_end)[0][0])


print(np.where(DateTimeCol_num == date_val_start)[0][0])
print(np.where(DateTimeCol_num == date_val_end)[0][0])

print(np.where(DateTimeCol_num == date_test_start)[0][0])
print(np.where(DateTimeCol_num == date_test_end)[0][0])

52603
70122
70123
78882
78883
96403


In [ ]:
for i in range(len(DateTimeCol)):
    date = DateTimeCol[i]
    if int(date.year == 2017) and int(date.month) == 1 and int(date.day) == 1 and int(date.hour) == 0: # 
        print("index for Jan 1, 2017: ", i)
    if int(date.year) == 2018 and int(date.month) == 12 and int(date.day) == 31 and int(date.hour) == 22:
        print("index for Dec 31, 2021: ", i)
    if int(date.year) == 2019 and int(date.month) == 1 and int(date.day) == 1 and int(date.hour) == 0: # 
        print("index for Jan 1, 2019: ", i)
    if int(date.year) == 2019 and int(date.month) == 12 and int(date.day) == 31 and int(date.hour) == 22:
        print("index for Dec 31, 2019 ", i)
    if int(date.year) == 2020 and int(date.month) == 1 and int(date.day) == 1 and int(date.hour) == 0: # 
        print("index for Jan 1, 2019: ", i)
    if int(date.year) == 2021 and int(date.month) == 5 and int(date.day) == 31 and int(date.hour) == 22: 
        print("index for Dec 31, 2019 ", i)
        break

index for Jan 1, 2017:  52603
index for Dec 31, 2021:  70121
index for Jan 1, 2019:  70123
index for Dec 31, 2019  78881
index for Jan 1, 2019:  78883
index for Dec 31, 2019  91289


In [6]:
def return_sequences(data, outputData, input_n_steps, output_n_steps):
    X = []
    Y = []
    length = len(data)
    print(length)
    for i in range(0,length, 1):
        input_indx = i + input_n_steps
        #output_indx = input_indx + output_n_steps
        output_indx = i + output_n_steps
        if (input_indx >= len(data)): # we need to have equally split sequences.
            break               # The remaining data that cannot fit into a fixed
                                # sequence will immediately be cut!
        else:
            Xsample = data.iloc[i:input_indx, :] # get the previous data
            #Ysample = outputData[input_indx:output_indx]
            Ysample = outputData[i]
            X.append(Xsample)
            Y.append(Ysample)
    X = np.asarray(X).astype('float64')
    Y = np.asarray(Y).astype('float64')
    return (X, Y)

In [7]:
from sklearn.preprocessing import StandardScaler 
def splitDataAndScale(data, output, train_start = None, train_end = None, val_start = None, val_end = None, test_start = None, test_end = None):
    #train_end +=1
    #val_end += 1
    #test_end += 1
    TrainingData = (data.iloc[train_start: train_end + 1, :].copy())
    print(len(TrainingData))
    TrainingCategories = TrainingData.iloc[:, [0,1]]
    TrainingNumerical = TrainingData.iloc[:, 2:]
    TrainingOutput = output[train_start + 6: train_end + 2].copy()  
    Scaler = StandardScaler().fit(TrainingNumerical)
    TrainingNumerical = Scaler.transform(TrainingNumerical)
    TrainingCategories = TrainingCategories.reset_index(drop = True)
    TrainingData = pd.concat([TrainingCategories, pd.DataFrame(TrainingNumerical)], axis = 1)
    TrainingData.reset_index(drop = True, inplace = True)
    TrainingOutput.reset_index(drop = True, inplace = True)
    
    ValidationData = data.iloc[val_start: val_end + 1, :].copy()
    ValidationData.reset_index(drop = True, inplace = True)
    ValidationCategories = ValidationData.iloc[:, [0,1]]
    ValidationNumerical = ValidationData.iloc[:, 2:]
    ValidationNumerical = Scaler.transform(ValidationNumerical)
    ValidationCategories = ValidationCategories.reset_index(drop = True)
    ValidationData = pd.concat([ValidationCategories, pd.DataFrame(ValidationNumerical)], axis = 1)
    ValidationOutput = output[val_start + 6: val_end + 2].copy()
    ValidationData.reset_index(drop = True, inplace = True)
    ValidationOutput.reset_index(drop = True, inplace = True)
    
    TestingData = data.iloc[test_start: test_end + 1, :].copy()
    TestingData.reset_index(drop = True, inplace = True)
    TestingCategories = TestingData.iloc[:, [0,1]]
    TestingNumerical = TestingData.iloc[:, 2:]
    TestingNumerical = Scaler.transform(TestingNumerical)
    TestingCategories = TestingCategories.reset_index(drop = True)
    TestingData = pd.concat([TestingCategories, pd.DataFrame(TestingNumerical)], axis = 1)
    TestingOutput = output[test_start + 6: test_end + 2].copy()
    TestingData.reset_index(drop = True, inplace = True)
    TestingOutput.reset_index(drop = True, inplace = True)


    TrainingSequences = return_sequences(TrainingData, TrainingOutput, 6, 1)

    TransformedTrainingData = TrainingSequences[0]
    TransformedTrainingOutput = TrainingSequences[1]

    ValidationSequences = return_sequences(ValidationData, ValidationOutput, 6, 1)

    TransformedValidationData = ValidationSequences[0]
    TransformedValidationOutput = ValidationSequences[1]

    TestingSequences = return_sequences(TestingData, TestingOutput, 6, 1)

    TransformedTestingData = TestingSequences[0]
    TransformedTestingOutput = TestingSequences[1]


    return (TransformedTrainingData, TransformedTrainingOutput, TransformedValidationData, TransformedValidationOutput,
    TransformedTestingData, TransformedTestingOutput)


In [8]:
from sklearn.preprocessing import StandardScaler 

DemandData = WeatherData['Demand'].copy() # The output data
WeatherData.drop(columns = ['Demand'], inplace = True)
data = splitDataAndScale(WeatherData, DemandData, train_start = 52603, train_end = 70121, val_start = 70123, val_end = 78881, test_start = 78883, test_end = 91289) # splitting the data into training, validation, and testing.


TrainingData = data[0]
TrainingOutput = data[1]

ValidationData = data[2]
ValidationOutput = data[3]

TestingData = data[4]
TestingOutput = data[5]

17519
17519
8759
12407


In [9]:
print(TrainingData.shape)
print(ValidationData.shape)
print(TestingData.shape)

(17513, 6, 12)
(8753, 6, 12)
(12401, 6, 12)


In [10]:
from torch.nn.utils.rnn import PackedSequence
from typing import *
import torch.nn as nn 


class LSTMModel(torch.nn.Module):
    def __init__(self, input_size, params1, params2, params3, num_layers, output_size):    
    #params1 = [conv_kernel_size, stride, max_kernel_size, LSTM_hidden_size, LSTM_num_layers]
    #params2 = [dropout_LSTM, dropout_FFN]
    #params3 = [hidden_size1, hidden_size2, hidden_size3]
        super(LSTMModel, self).__init__()        
        self.conv1D = torch.nn.Conv1d(in_channels = 18, out_channels = 18, kernel_size = params1[0], stride = params1[1])
        self.max1D = torch.nn.MaxPool1d(kernel_size = params1[2], stride = params1[1])
        self.dropout1 = torch.nn.Dropout(params2[0])
        self.input_size = input_size - params1[0] - params1[2] + 2
        self.BiLSTM = nn.LSTM(input_size = self.input_size, hidden_size = params1[3], bidirectional = False, num_layers = params1[4])
        
        layers = []
        input_size = params1[3] #params1[3] *2 
        num_units = 0 
        for i in range(num_layers):
            num_units = params3[i] 
            layers.append(torch.nn.Linear(input_size, num_units, bias = True))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(params2[0])) #also add in the dropout. 
            input_size = num_units
        
        self.intermediate_layers = torch.nn.Sequential(*layers)
        self.finalNN = torch.nn.Linear(num_units, output_size, bias = True)
        
    def forward(self, x):
        x = self.conv1D(x)
        x = self.max1D(x) 
        x = self.dropout1(x) 
        x = self.BiLSTM(x) 
        
        x = x[0]
        x = x[:, -1,:]
        #x = x[1][0]
        
        x = self.intermediate_layers(x) 
        x = self.finalNN(x) 
        return x 


In [12]:
model = torch.load("/home/jik19004/FilesToRun/BayesianBiDirectional/Long_Models/LSTMLong")
print(model)

LSTMModel(
  (conv1D): Conv1d(18, 18, kernel_size=(3,), stride=(1,))
  (max1D): MaxPool1d(kernel_size=3, stride=1, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.15, inplace=False)
  (BiLSTM): LSTM(8, 64)
  (intermediate_layers): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.15, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.15, inplace=False)
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.15, inplace=False)
  )
  (finalNN): Linear(in_features=16, out_features=1, bias=True)
)


In [ ]:
no

In [11]:
import optuna
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class TimeSeriesDataset(Dataset):
    def __init__(self, data, output):
        data = torch.tensor(data).float();
        output = torch.tensor(output).float()
        self.data = data
        self.output = output;

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx];
        y = self.output[idx];

        return x, y;

# use the past 72 hours in advance and then predict the 1st hour, 6th hour, 12 hours!

def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for data, target in val_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item() * target.size(0)
    return running_loss / len(val_loader.dataset)

def Train_and_Evaluate(train_loader, val_loader, device, params1, params2, params3, numEpochs, early_stop_epochs):
    model = LSTMModel(input_size = 12, params1 = params1, params2 = params2, params3 = params3, num_layers = 3, output_size = 1)
    model = model.to(device);
    LossFunction = torch.nn.L1Loss();
    best_val_loss = float('inf')
    Training_Loss = float('inf')
    early_stop_count = 0


    Optimizer = torch.optim.Adam(params = model.parameters())
    for epoch in range(0,numEpochs):
        model.train()
        Training_Loss = 0;
        total_samples = 0;
        for input, output in train_loader:
            input = input.to(device);
            #output = torch.squeeze(output, 1);
            output = output.to(device);
            predictedVal = model(input)
            predictedVal = torch.squeeze(predictedVal, 1)
            Optimizer.zero_grad();
            batchLoss = LossFunction(predictedVal, output);
            batchLoss.backward();
            Optimizer.step();
            Training_Loss += batchLoss * output.size(0) #* output.size(0);
            total_samples += output.size(0)
        Training_Loss = Training_Loss.item()/total_samples


        Validation_Loss = 0;
        print("passed ", epoch, "epoch", "Training Loss: ", Training_Loss," ", end = "")
        with torch.no_grad(): 
            model.eval() 
            total_val_samples = 0 
            Validation_Loss = 0 
            for val_input, val_output in val_loader:
                val_input = val_input.to(device)
                #val_output = torch.squeeze(val_output,1);
                val_output = val_output.to(device)
                predictedVal = model(val_input)
                predictedVal = torch.squeeze(predictedVal, 1)
                Validation_Loss += LossFunction(val_output, predictedVal) * val_output.size(0)
                total_val_samples += val_output.size(0)
            Validation_Loss = Validation_Loss/total_val_samples
            print("Validation Loss: ", Validation_Loss)

            if Validation_Loss < best_val_loss:
                best_val_loss = Validation_Loss
                torch.save(model, "LSTMLong")
                early_stop_count = 0;   
            else:
                early_stop_count += 1
            if early_stop_count >= early_stop_epochs:
                return (Training_Loss, best_val_loss)
    return (Training_Loss, best_val_loss)

def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for data, target in val_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item() * target.size(0)
    return running_loss / len(val_loader.dataset)


def evaluate2(model, val_loader, criterion, criterion2, device):
    num_experiments = 1
    criterion2 = criterion2.to(device)
    model.eval()

    with torch.no_grad():
        total_val_samples = 0;
        Validation_Loss_MAE = 0;
        Validation_Loss_MAPE = 0;
        predictions = []  
        std_list = [] 
        for val_input, val_output in val_loader:
            val_input = val_input.to(device);
            val_output = val_output.to(device);
            #Avgpred
            pred = model(val_input)
            pred = torch.squeeze(pred, 1)
            pred = torch.unsqueeze(pred, 0)
               
            for i in range(num_experiments - 1):
                predictedVal2 = model(val_input)
                predictedVal2 = torch.squeeze(predictedVal2, 1)
                predictedVal2 = torch.unsqueeze(predictedVal2, 0)
                pred = torch.cat([pred, predictedVal2], dim = 0)
            
            Avgpred = torch.mean(pred, dim = 0)
            stdev = torch.std(pred, dim = 0)
            predCsv = Avgpred.cpu().numpy()
            stdev = stdev.cpu().numpy()
            predictions.extend(predCsv) 
            
            Validation_Loss_MAE += criterion(val_output, Avgpred) * val_output.size(0)
            Validation_Loss_MAPE += criterion2(val_output, Avgpred) * val_output.size(0)
            total_val_samples += val_output.size(0)
        Validation_Loss_MAE = Validation_Loss_MAE/total_val_samples
        Validation_Loss_MAPE = Validation_Loss_MAPE/total_val_samples 
        return Validation_Loss_MAE, Validation_Loss_MAPE, predictions   

In [12]:
def Train_and_Evaluate2(train_loader, val_loader, device, params1, params2, numEpochs, early_stop_epochs):
    #num_layers, input_dim, hidden_unit1, hidden_unit2, output_unit, lastNeurons, batch_size, params, device = None
    model = LSTMModel(params1 = params1, params2 = params2, device = device)
    model = model.to(device);
    TrainEpochLoss = [] 
    ValidationEpochLoss = [] 
    LossFunction = torch.nn.L1Loss();
    best_val_loss = float('inf')
    early_stop_count = 0


    Optimizer = torch.optim.Adam(params = model.parameters())
    for epoch in range(0,numEpochs):
        model.train()
        Training_Loss = 0;
        total_samples = 0;
        for input, output in train_loader:
            input = input.to(device);
            output = torch.squeeze(output, 1);
            output = output.to(device);
            predictedVal = model(input)
            predictedVal = torch.squeeze(predictedVal, 1)
            Optimizer.zero_grad();
            batchLoss = LossFunction(predictedVal, output);
            batchLoss.backward();
            Optimizer.step();
            Training_Loss += batchLoss * output.size(0) #* output.size(0);
            total_samples += output.size(0)
        Training_Loss = Training_Loss.item()/total_samples
        TrainEpochLoss.append(Training_Loss)


        Validation_Loss = 0;
        print("passed ", epoch, "epoch", "Training Loss: ", Training_Loss," ", end = "")
        with torch.no_grad():
            model.eval()
            total_val_samples = 0;
            Validation_Loss = 0;
            for val_input, val_output in val_loader:
                val_input = val_input.to(device);
                val_output = torch.squeeze(val_output,1);
                val_output = val_output.to(device);
                predictedVal = model(val_input)
                predictedVal = torch.squeeze(predictedVal, 1)
                Validation_Loss += LossFunction(val_output, predictedVal) * val_output.size(0)
                total_val_samples += val_output.size(0)
            Validation_Loss = Validation_Loss.item()/total_val_samples
            print("Validation Loss: ", Validation_Loss)
            ValidationEpochLoss.append(Validation_Loss)

            if Validation_Loss < best_val_loss:
                best_val_loss = Validation_Loss
                torch.save(model, "LSTMLong")
                early_stop_count = 0;   
            else:
                early_stop_count +=1
            if early_stop_count >= early_stop_epochs:
                return (TrainEpochLoss, ValidationEpochLoss);
    return (TrainEpochLoss, ValidationEpochLoss);


In [13]:
TrainingDataset = TimeSeriesDataset(np.array(TrainingData),np.array(TrainingOutput));
TrainingLoader = DataLoader(TrainingDataset, batch_size = 256);


ValidationDataset = TimeSeriesDataset(ValidationData, ValidationOutput); ### Set it with the previous validation data
ValidationLoader = DataLoader(ValidationDataset, batch_size = 256);


TestingDataset = TimeSeriesDataset(TestingData,TestingOutput); ### Set it with the previous testing data.
TestingLoader = DataLoader(TestingDataset, batch_size = 256);

In [14]:
from torchmetrics.regression import MeanAbsolutePercentageError

def LongTermEvaluate(model, ValidationData, ValidationOutput, DateTimeCol = DateTimeCol, index_start = 70117, index_end = 91291):
    Validation_Loss_MAE = []
    Validation_Loss_MAPE = []
    df_val = pd.DataFrame() 
    df_val_std = pd.DataFrame()
    
    for i in range(index_start, index_end, 24):       
        val_start = i
        val_end = val_start + 28 #val_start + 28
        val_output_start = DateTimeCol[val_start + 6]
        val_output_end = DateTimeCol[val_end + 1]

        if val_end < index_end:            
            ValidationDataset = TimeSeriesDataset(np.array(ValidationData[val_start - index_start: val_start - index_start + 24]), 
                                               np.array(ValidationOutput[val_start - index_start: val_start - index_start + 24]))
            ValidationLoader = DataLoader(ValidationDataset, batch_size = 3, shuffle = False) 
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


            val_str = val_output_start + "-" + val_output_end           
            print(val_str) 
            val_loss = evaluate2(model, ValidationLoader, torch.nn.L1Loss(),MeanAbsolutePercentageError(), device)            
            Validation_Loss_MAE.append(val_loss[0].item())
            Validation_Loss_MAPE.append(val_loss[1].item())
            df_val = pd.concat([df_val, pd.DataFrame({val_str: val_loss[2]})], ignore_index=False, axis=1)
            df_val_std = pd.concat([df_val_std, pd.DataFrame({val_str: val_loss[3]})], ignore_index = False, axis =1 )
        else:
            return Validation_Loss_MAE, Validation_Loss_MAPE, df_val, df_val_std 
         
#model = torch.load("/home/jik19004/FilesToRun/BayesianBiDirectional/BayesianBiLSTMLong")

WeatherData = pd.read_csv('/home/jik19004/FilesToRun/ASOS_10_CT_stations_tmpc_demand_2011_2023.csv').drop(columns = ["Unnamed: 0"])
DateTimeCol = WeatherData["Datetime"]

"""tuples = LongTermEvaluate(model, ValidationData, ValidationOutput, DateTimeCol = DateTimeCol, index_start = 70123, index_end = 72000)

ValidationLoss_series = pd.DataFrame({"Validation_MAE": tuples[0], "Validation_MAPE": tuples[1]})
ValidationLoss_series.to_csv("/home/jik19004/FilesToRun/BayesianBiDirectional/ValidationLossesLong.csv", index = False)
tuples[2].to_csv("/home/jik19004/FilesToRun/BayesianBiDirectional/ValidationPredictionsLong.csv", index = False)
tuples[3].to_csv("/home/jik19004/FilesToRun/BayesianBiDirectional/ValidationStdLong.csv", index = False)"""

'tuples = LongTermEvaluate(model, ValidationData, ValidationOutput, DateTimeCol = DateTimeCol, index_start = 70123, index_end = 72000)\n\nValidationLoss_series = pd.DataFrame({"Validation_MAE": tuples[0], "Validation_MAPE": tuples[1]})\nValidationLoss_series.to_csv("/home/jik19004/FilesToRun/BayesianBiDirectional/ValidationLossesLong.csv", index = False)\ntuples[2].to_csv("/home/jik19004/FilesToRun/BayesianBiDirectional/ValidationPredictionsLong.csv", index = False)\ntuples[3].to_csv("/home/jik19004/FilesToRun/BayesianBiDirectional/ValidationStdLong.csv", index = False)'

In [18]:
data = pd.read_csv("../TestingLong/NormalTestingLossesLong.csv")
data2 = pd.read_csv("../TestingLong/TestingLossesLong.csv")
print(np.average(data["Testing_MAPE"]))
print(np.average(data2["Testing_MAPE"]))

0.0936849250696426
0.049352038265458566
